## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2020
month = 11
program = "default"
study_type = "program"
mode_of_interest = "e-bike"
include_test_users = False
is_debug_mode = True
dynamic_labels = {}

In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd

from plots import *
import scaffolding

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
# Do not run this notebook at all unless it is for a program; nbclient will run up through this cell
if study_type != "program":
    ipython = get_ipython()
    ipython._showtraceback = scaffolding.no_traceback_handler
    raise Exception("The plots in this notebook are only relevant to programs")

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r df_ei
%store -r dic_re
%store -r dic_pur
%store -r dic_fuel

# convert a dictionary to a defaultdict
dic_re = defaultdict(lambda: 'Other',dic_re)
dic_pur = defaultdict(lambda: 'Other',dic_pur)
dic_fuel = defaultdict(lambda: 'Other',dic_fuel)

## Collect Data From Database

In [ ]:
expanded_ct, file_suffix, quality_text, debug_df = scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            dynamic_labels,
                                                                            dic_re,
                                                                            dic_pur=dic_pur,
                                                                            include_test_users=include_test_users)
# CASE 1 of https://github.com/e-mission/em-public-dashboard/issues/69#issuecomment-1256835867 
expanded_ct = scaffolding.add_energy_impact(expanded_ct, df_ei, dic_fuel, dynamic_labels) if len(expanded_ct) > 0 else expanded_ct

In [ ]:
if 'mode_confirm' in expanded_ct.columns:
    mode_of_interest_df = expanded_ct.query(f"mode_confirm == '{mode_of_interest}'")
    debug_df.loc[f"{mode_of_interest}_trips"] = len(mode_of_interest_df)
    debug_df.loc[f"{mode_of_interest}_trips_with_replaced_mode"] = scaffolding.trip_label_count("Replaced_mode", mode_of_interest_df)

## Energy Impacts Relative to Specific Mode

In [ ]:
# CASE 2 of https://github.com/e-mission/em-public-dashboard/issues/69#issuecomment-1256835867
data_eb = expanded_ct.query(f"mode_confirm == '{mode_of_interest}'") if "mode_confirm" in expanded_ct.columns else expanded_ct

In [ ]:
quality_text_mode = scaffolding.get_quality_text(expanded_ct, data_eb, mode_of_interest)

### Sketch of energy impact by trips for specified mode

In [ ]:
# ebei : ebike energy impact
plot_title_no_quality=f"Sketch of Energy Impact of {mode_of_interest} trips"
file_name =f'sketch_energy_impact_{mode_of_interest}%s' % file_suffix
    
try:
    ebei=data_eb.groupby('Replaced_mode').agg({'Energy_Impact(kWH)': ['sum', 'mean']},)
    ebei.columns = ['Sketch of Total Energy_Impact(kWH)', 'Sketch of Average Energy_Impact(kWH)']
    ebei= ebei.reset_index()
    ebei = ebei.sort_values(by=['Sketch of Total Energy_Impact(kWH)'], ascending=False)
    ebei['boolean'] = ebei['Sketch of Total Energy_Impact(kWH)'] > 0
    net_energy_saved = round(sum(ebei['Sketch of Total Energy_Impact(kWH)']), 2)

    x = ebei['Sketch of Total Energy_Impact(kWH)']
    y = ebei['Replaced_mode']
    color =ebei['boolean']

    plot_title= plot_title_no_quality+f"\n Contribution by replaced mode towards a total of {net_energy_saved}(kWH)\n"+quality_text
    energy_impact(x,y,color,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(x.values,y), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Sketch of CO2 emissions impact by trips of specified mode

In [ ]:
#  Case: label_options is present on the config file
if len(dynamic_labels) > 0:
    plot_title_no_quality=f"Sketch of Total Kilograms of CO2 Emissions of {mode_of_interest} trips"
    file_name =f'sketch_CO2impact_{mode_of_interest}%s' % file_suffix

    try:
        ebco2=data_eb.groupby('Replaced_mode').agg({'CO2_Impact(kg)': ['sum', 'mean']},)
        ebco2.columns = ['total_kg_CO2_emissions', 'average_kg_CO2_emission']
        ebco2 = ebco2.reset_index()
        ebco2 = ebco2.sort_values(by=['total_kg_CO2_emissions'], ascending=False)
        ebco2['boolean'] = ebco2['total_kg_CO2_emissions'] > 0
        net_CO2_emissions = round(sum(ebco2['total_kg_CO2_emissions']), 2)

        x = ebco2['total_kg_CO2_emissions']
        y = ebco2['Replaced_mode']
        xLabel = 'CO2 Emissions (kg)'
        yLabel = 'Replaced Mode'
        color = ebco2['boolean']

        plot_title= plot_title_no_quality+f"\n Contribution by replaced mode towards a total of {net_CO2_emissions}(kg CO2 Emissions )\n"+quality_text
        CO2_impact(x,y,color,plot_title, xLabel,yLabel,file_name)
        alt_text = store_alt_text_bar(pd.DataFrame(x.values,y), file_name, plot_title)
        
        if (is_debug_mode == True):
            ebco2_lb=data_eb.groupby('Replaced_mode').agg({'CO2_Impact(lb)': ['sum', 'mean']},)
            ebco2_lb.columns = ['total_lb_CO2_emissions', 'average_lb_CO2_emission']
            ebco2_lb = ebco2_lb.reset_index()
            ebco2_lb = ebco2_lb.sort_values(by=['total_lb_CO2_emissions'], ascending=False)
            
            scaffolding.print_CO2_emission_calculations(data_eb, ebco2_lb, ebco2, dynamic_labels)
     
    except:
        generate_missing_plot(plot_title_no_quality,debug_df,file_name)
        alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)
# case: when label_options is absent from the config file
else: 
    plot_title_no_quality=f"Sketch of Total pounds of CO2 Emissions of {mode_of_interest} trips"
    file_name =f'sketch_CO2impact_{mode_of_interest}%s' % file_suffix

    try:
        ebco2=data_eb.groupby('Replaced_mode').agg({'CO2_Impact(lb)': ['sum', 'mean']},)
        ebco2.columns = ['total_lb_CO2_emissions', 'average_lb_CO2_emission']
        ebco2 = ebco2.reset_index()
        ebco2 = ebco2.sort_values(by=['total_lb_CO2_emissions'], ascending=False)
        ebco2['boolean'] = ebco2['total_lb_CO2_emissions'] > 0
        net_CO2_emissions = round(sum(ebco2['total_lb_CO2_emissions']), 2)

        x = ebco2['total_lb_CO2_emissions']
        y = ebco2['Replaced_mode']
        xLabel = 'CO2 Emissions (lb)'
        yLabel = 'Replaced Mode'
        color = ebco2['boolean']

        plot_title= plot_title_no_quality+f"\n Contribution by replaced mode towards a total of {net_CO2_emissions}(lb CO2 Emissions )\n"+quality_text
        CO2_impact(x,y,color,plot_title,xLabel,yLabel,file_name)
        alt_text = store_alt_text_bar(pd.DataFrame(x.values,y), file_name, plot_title)

        if (is_debug_mode == True):
            ebco2_kg=data_eb.groupby('Replaced_mode').agg({'CO2_Impact(kg)': ['sum', 'mean']},)
            ebco2_kg.columns = ['total_kg_CO2_emissions', 'average_kg_CO2_emission']
            ebco2_kg = ebco2_kg.reset_index()
            ebco2_kg = ebco2_kg.sort_values(by=['total_kg_CO2_emissions'], ascending=False)
            
            scaffolding.print_CO2_emission_calculations(data_eb, ebco2, ebco2_kg, dynamic_labels)
    except:
        generate_missing_plot(plot_title_no_quality,debug_df,file_name)
        alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)